<a href="https://colab.research.google.com/github/grifire/tp-initiation-llm-student-version/blob/main/mini_projet_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mini projet : Système de Questions-Réponses avec Classement de Qualité

**Objectif** : Créer un système qui utilise un LLM pour répondre à des questions sur un domaine spécifique et évaluer la qualité des réponses.

**Durée estimée** : 3-4 heures

**Description du projet** :
1. **Créer un dataset de questions-réponses** : Définir 10-15 questions sur un domaine de votre choix (technologie, cinéma, histoire, etc.) avec leurs réponses de référence
2. **Tester différentes approches de prompt** :
    - Zero-shot
    - One-shot
    - Few-shot
3. **Comparer les paramètres de génération** : Tester différentes combinaisons de température, top_k, top_p pour voir leur impact
4. **Évaluation automatique** : Créer une fonction qui compare la réponse du LLM avec la réponse de référence (vous pouvez utiliser des métriques simples comme la longueur, les mots-clés communs, etc.)
5. **Visualisation des résultats** : Afficher un tableau comparatif des performances selon les différentes approches

**Livrables attendus** :
- Code documenté avec des commentaires
- Analyse des résultats en markdown (quelle approche fonctionne le mieux ?)
- Au moins 2 visualisations (graphiques ou tableaux)

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# model_name = "microsoft/phi-2"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
"""
dataset est une liste de question/réponse qui serviront de référence pour la réponse du llm
"""
dataset = [
{"ask": "Quelle est la vitesse de la lumière dans le vide ?",
 "answer": "Environ 299 792 kilomètres par seconde."},

{"ask": "Quel est le nom de notre galaxie ?",
 "answer": "La Voie lactée."},

{"ask": "Quel est l’âge approximatif de l’Univers ?",
 "answer": "Environ 13,8 milliards d’années."},

{"ask": "Qu’est-ce qu’un trou noir ?",
 "answer": "Un objet céleste dont la gravité est si forte que rien, pas même la lumière, ne peut s’en échapper."},

{"ask": "Quelle planète est la plus proche du Soleil ?",
 "answer": "Mercure."},

{"ask": "Quelle planète est connue comme la planète rouge ?",
 "answer": "Mars."},

{"ask": "Qu’est-ce qu’une étoile filante ?",
 "answer": "Un météore brûlant en entrant dans l’atmosphère terrestre."},

{"ask": "Quel est le plus grand planet du système solaire ?",
 "answer": "Jupiter."},

{"ask": "Comment s’appelle le satellite naturel de la Terre ?",
 "answer": "La Lune."},

{"ask": "Qu’est-ce qu’une galaxie ?",
 "answer": "Un ensemble gigantesque d’étoiles, de gaz, de poussières et de matière noire."},

{"ask": "Quelle est la forme approximative de la Voie lactée ?",
 "answer": "Une galaxie spirale barrée."},

{"ask": "Qu’est-ce qu’une supernova ?",
 "answer": "L’explosion spectaculaire d’une étoile massive en fin de vie."},

{"ask": "De quoi est principalement composée l’atmosphère du Soleil ?",
 "answer": "Principalement d’hydrogène et d’hélium."},

{"ask": "Comment appelle-t-on l’ensemble des planètes en orbite autour d’une étoile ?",
 "answer": "Un système planétaire."},

{"ask": "Qu’est-ce que l’ISS ?",
 "answer": "La Station spatiale internationale, un laboratoire orbital habité en permanence."}
]


In [ ]:
"""
generate_response prend en entrée un prompt et retourne la réponse générée par le modèle
"""
def generate_response(prompt,afficher_prompt = False):
  if afficher_prompt :
    print(prompt)
    print("===================================")
  inputs = tokenizer(prompt, return_tensors='pt') # retourner les tenseurs
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens=50, # max 50 tokens générés
      )[0],
      skip_special_tokens=True # on ne génère pas les tokens spéciaux <, >, ...
  )
  return output
dialogue=dataset[0]
# print(dialogue["ask"])
print(generate_response(dialogue["ask"]))

## Zero Shot

In [ ]:
"""
ZeroShot génère un prompte simple dans le but d'aider le LLM à répondre à la question
"""
def ZeroShot(question):
  prompt = f"Répond à la question suivante : \n {question} \n réponse : "
  return prompt

dialogue=dataset[2]
print(generate_response(ZeroShot(dialogue["ask"]), True))
#

## One Shot

In [ ]:
"""
OneShot Génère un exemple de question réponse pour faire au LLM une démonstration
de comment générer une réponse.
"""
def OneShot(question, ds, index,afficher_prompt = False):
  ref = ds[index]
  prompt = f"""
Question :

{ref['ask']}


Réponse :
{ref['answer']}


Question :

{question}

Réponse :"""

  return generate_response(prompt, afficher_prompt)

dialogue=dataset[2]
print(OneShot(dialogue["ask"], dataset, 0, True))
#

## Multi Shot

In [ ]:
"""
make_prompt est la version multi shot du OneShot.
il prend en entrée une question, un dataset, et une liste d'index du dataset
"""
def make_prompt(question, ds, indexs = [0]) :
  prompt = "";
  for i in indexs :
    ref = ds[i]
    prompt += f"""
Question :

{ref['ask']}

Réponse :
{ref['answer']}


"""
  prompt += f"""
Question :

{question}

Réponse :
"""
  return prompt

question = "Quelle est la superficie de la Terre?"
print(generate_response(make_prompt(question, dataset,[0,1,2,3,4,5,6]), True))

In [ ]:
"""
evaluation permet d'évalué un prompte avec la réponse du LLM et la réponse attendu.
Il génère un tabeau de 3 valeurs : le nombre de mot de la réponse, le nombre de mot attendu,
et le nombre en commun entre la réponse du LLM et celle du LLM
"""
def evaluation(prompt,reponse,attendu) :

  print(f"Prompte : {prompt}")
  print(f"Réponse : {reponse.replace(prompt,"")}")
  print(f"Réponse attendue : {attendu}")
  nb_mots = len(reponse.split())
  nb_mots_ref = len(attendu.split())
  nb_mots_communs = len(set(reponse.split()).intersection(set(attendu.split())))
  tableau = f"""
  Nombre de mots obtenu \tNombre de mots réponse attendue \tNombre de mots en commun
  {nb_mots}\t\t\t\t{nb_mots_ref}\t\t\t\t\t{nb_mots_communs}
===============================================================================================================
  """
  print(tableau)

In [ ]:
print("Tableau de comparaison")
print("Zero Shot")
evaluation(ZeroShot(dataset[2]["ask"]), generate_response(ZeroShot(dataset[2]["ask"])), dataset[2]["answer"])
print("One Shot")
evaluation(make_prompt(question, dataset,[0]), generate_response(make_prompt(question, dataset,[0])), dataset[2]["answer"])
print("Multi Shot (x2)")
evaluation(make_prompt(question, dataset,[0,1]), generate_response(make_prompt(question, dataset,[0,1])), dataset[2]["answer"])
print("Multi Shot (x7)")
evaluation(make_prompt(question, dataset,[0,1,2,3,4,5,6]), generate_response(make_prompt(question, dataset,[0,1,2,3,4,5,6])), dataset[2]["answer"])

## Conclusion Partie 2 Zero/One/Multi shot

Le Zero shot semble être la solution la plus adapté car le llm à tendance a vouloir poursuivre le schéma et génère de nouveau de question/réponse après avoir répondu.
Avec un prompte Zero shot il répond simplement à la question

In [ ]:
def generate_response_plus(temperature, top_k, top_p, sampling, prompt,attendu):
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
            top_k=top_k,
            top_p=top_p,
            do_sample=sampling,

        )[0],
        skip_special_tokens=True
    )
    print(f"T={temperature}, top_k={top_k}, top_p={top_p}, sampling={sampling}\n") #.replace(prompt,"")
    evaluation(prompt,output,attendu)
    print()


prompt = dataset[2]["ask"]
prompt = ZeroShot(prompt)
generate_response_plus(1, 50, 1, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 1, False, prompt, dataset[2]["answer"])
generate_response_plus(50, 50, 1, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.1, True, prompt, dataset[2]["answer"])
generate_response_plus(200, 50, 0.1, True, prompt, dataset[2]["answer"])
generate_response_plus(50, 3, 0.7, True, prompt, dataset[2]["answer"])
generate_response_plus(10, 30, 5, True, prompt, dataset[2]["answer"])
generate_response_plus(10, 30, 5, False, prompt, dataset[2]["answer"])
generate_response_plus(200, 600, 1, True, prompt, dataset[2]["answer"])
generate_response_plus(200, 600, 0.99, True, prompt, dataset[2]["answer"])
generate_response_plus(200, 600, 0.99, False, prompt, dataset[2]["answer"])

print(f"\n\nRéponse attendu :\n{dataset[2]["answer"]}")

In [ ]:
generate_response_plus(1, 50, 0.1, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.2, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.5, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.7, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.8, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 0.99, True, prompt, dataset[2]["answer"])
generate_response_plus(1, 50, 1, True, prompt, dataset[2]["answer"])

## Les paramêtre :
Sampling : Autorise le LLM à être inventif (Vrai s'il doit être inventif). Les paramêtres suivants ne sont pas impactant si il est réglé sur Faux.

Temperature : Le degrés d'inventivité. Plus il est élevé, plus la réponses obtenu risque d'être hors sujet ou chaotique.

top_k : le nombre de token qu'il peut prendre en considération.

top_p : Si il est plus petit que 1, il permet d'influencé sur la réponse. Plus il est proche de 1 plus la réponse obtenu risque de fluctuer. Contrairement à la température, il n'influe pas sur la logique de la réponse même si celle-si est fausse.
